# AMLO Exploratory Data Analysis

In [5]:
import os
import pandas as pd
from tqdm import tqdm
import torch


from amlo_parser import AMLOParser
from training_set import TrainingSet

#### CONSTANTS

In [3]:
PATH = "C:/Users/fdmol/Desktop/AMLO-NLP/src/data/text_files/"
LABELED_PATH = "C:/Users/fdmol/Desktop/AMLO-NLP/src/data/amlo_labeling.xlsx"

### Pipeline

Create training set, along with its correspoding txt files

In [4]:
all_files = os.listdir(PATH)

training_set = TrainingSet(remove_stopwords=True)
training_set.create_training_set()

100%|██████████| 1246/1246 [00:00<00:00, 6523.12it/s]

Conference 20181207 is not agressive
Conference 20190102 is not agressive
Conference 20190111 is not agressive
Conference 20190227 is not agressive
Conference 20200128 is not agressive
Conference 20210510 is not agressive
Conference 20221125 is not agressive


### Model training for scoring

### Trying to implement a NNet model

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from torch.nn.utils.rnn import pad_sequence


class TextDataset(Dataset):
    def __init__(self, texts, scores):
        self.texts = texts
        self.scores = scores

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {"text": self.texts[idx], "score": self.scores[idx]}


def numericalize(text):
    return [vocab[word] for word in tokenizer(text) if word in vocab]

In [ ]:
# Assuming create_regression_training_df and create_unseen_df methods are called before
# Tokenize and numericalize text data
tokenizer = lambda x: x.split()  # Simple tokenizer
vocab = Counter()

for text in training_df["text"]:
    vocab.update(tokenizer(text))

vocab = {
    word: i + 1 for i, word in enumerate(vocab)
}  # Mapping word to index (starting from 1)


training_df["text"] = training_df["text"].apply(numericalize)
unseen_df["text"] = unseen_df["text"].apply(numericalize)

# Convert to PyTorch datasets
train_dataset = TextDataset(
    torch.tensor(list(training_df["text"]), dtype=torch.long),
    torch.tensor(training_df["score"].values, dtype=torch.float),
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)